# 📊 Trader Behavior vs Market Sentiment Analysis
## Web3 Trading Data Science Project

**Candidate:** Aswin  
**Objective:** Analyze how market sentiment (Fear & Greed Index) influences trader performance on Hyperliquid DEX

---

### 📂 Datasets
1. **Fear & Greed Index** - Daily Bitcoin market sentiment (2018-2025)
2. **Hyperliquid Trading Data** - Historical trades with P&L, volume, and leverage

### 🎯 Key Questions
- Do traders perform better during Fear or Greed periods?
- How does trading volume change with sentiment?
- Does risk appetite (leverage) vary with market conditions?

---

## 1️⃣ Setup & Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
from datetime import datetime

# Settings
warnings.filterwarnings('ignore')
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Create folders
os.makedirs('csv_files', exist_ok=True)
os.makedirs('outputs', exist_ok=True)

print("✅ Setup complete!")

In [ ]:
# Download datasets from Google Drive
import gdown

print("📥 Downloading datasets...\n")

# Download Fear & Greed Index
gdown.download('https://drive.google.com/uc?id=1PgQC0tO8XN-wqkNyghWc_-mnrYv_nhSf',
               'fear_greed_index.csv', quiet=False)

# Download Trading Data
gdown.download('https://drive.google.com/uc?id=1IAfLZwu6rJzyWKgBToqwSmmVYU6VbjVs',
               'historical_data.csv', quiet=False)

print("\n✅ Downloads complete!")

In [ ]:
# Load datasets
sentiment_df = pd.read_csv('fear_greed_index.csv')
trader_df = pd.read_csv('historical_data.csv', low_memory=False)

print(f"📊 Sentiment Data: {len(sentiment_df):,} days")
print(f"📊 Trading Data: {len(trader_df):,} trades")
print(f"\n✅ Data loaded successfully!")

---
## 2️⃣ Data Exploration

In [ ]:
# Preview sentiment data
print("📋 Fear & Greed Index Sample:")
print("="*60)
display(sentiment_df.head())

print("\n📋 Columns:", list(sentiment_df.columns))

In [ ]:
# Preview trading data
print("📋 Trading Data Sample:")
print("="*60)
display(trader_df.head())

print("\n📋 Columns:", list(trader_df.columns))

---
## 3️⃣ Data Preprocessing

In [ ]:
# Convert sentiment dates to datetime
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])

print(f"📅 Sentiment date range: {sentiment_df['date'].min().date()} to {sentiment_df['date'].max().date()}")
print("✅ Sentiment data ready!")

In [ ]:
# Find and convert timestamp column in trading data
time_columns = [col for col in trader_df.columns if 'time' in col.lower()]
print(f"⏱️ Time column found: {time_columns[0]}")

timestamp_col = time_columns[0]

# Convert to datetime (handling DD-MM-YYYY format)
trader_df['datetime'] = pd.to_datetime(trader_df[timestamp_col], format='mixed', dayfirst=True)
trader_df['date'] = trader_df['datetime'].dt.date
trader_df['date'] = pd.to_datetime(trader_df['date'])

print(f"📅 Trading date range: {trader_df['date'].min().date()} to {trader_df['date'].max().date()}")
print(f"✅ Trading data ready! ({len(trader_df):,} trades across {trader_df['date'].nunique():,} days)")

---
## 4️⃣ Aggregate Trading Data by Day

We need to convert individual trades into daily statistics to match the sentiment data (which is one value per day).

In [ ]:
# Find key columns
pnl_col = [col for col in trader_df.columns if 'pnl' in col.lower()][0]
leverage_col = [col for col in trader_df.columns if 'leverage' in col.lower()][0] if any('leverage' in col.lower() for col in trader_df.columns) else None

print(f"💰 P&L Column: {pnl_col}")
if leverage_col:
    print(f"⚡ Leverage Column: {leverage_col}")

In [ ]:
# Aggregate by date
print("📊 Aggregating trading data by date...\n")

# Build aggregation dictionary
agg_dict = {
    pnl_col: ['sum', 'mean'],  # Total P&L and average per trade
    'datetime': 'count'  # Number of trades
}

if leverage_col:
    agg_dict[leverage_col] = 'mean'  # Average leverage

# Aggregate
daily_stats = trader_df.groupby('date').agg(agg_dict).reset_index()

# Flatten column names
daily_stats.columns = ['date', 'total_pnl', 'avg_pnl_per_trade', 'trade_count']
if leverage_col:
    daily_stats.columns = ['date', 'total_pnl', 'avg_pnl_per_trade', 'trade_count', 'avg_leverage']

print(f"✅ Created daily statistics for {len(daily_stats):,} trading days")
display(daily_stats.head())

---
## 5️⃣ Merge Sentiment + Trading Data

In [ ]:
# Merge datasets
merged_df = pd.merge(
    sentiment_df[['date', 'value', 'classification']],
    daily_stats,
    on='date',
    how='inner'  # Keep only dates with both sentiment and trading data
)

print(f"📊 Merged Dataset:")
print(f"   Total days: {len(merged_df):,}")
print(f"   Date range: {merged_df['date'].min().date()} to {merged_df['date'].max().date()}")
print(f"\n✅ Merge successful!")

display(merged_df.head(10))

In [ ]:
# Save merged dataset
merged_df.to_csv('csv_files/merged_trading_sentiment.csv', index=False)
print("💾 Saved: csv_files/merged_trading_sentiment.csv")

---
## 6️⃣ Exploratory Data Analysis

Now let's analyze how trading behavior changes with market sentiment.

### 📊 Visualization 1: Sentiment Distribution

In [ ]:
# Sentiment distribution
sentiment_counts = merged_df['classification'].value_counts()

# Create visualization
plt.figure(figsize=(10, 6))
colors = ['#d32f2f', '#f57c00', '#fbc02d', '#7cb342', '#388e3c']
sentiment_counts.plot(kind='bar', color=colors[:len(sentiment_counts)])
plt.title('Market Sentiment Distribution (Trading Days)', fontsize=14, fontweight='bold')
plt.xlabel('Sentiment Category', fontsize=12)
plt.ylabel('Number of Days', fontsize=12)
plt.xticks(rotation=45, ha='right')

# Add value labels on bars
for i, v in enumerate(sentiment_counts):
    plt.text(i, v + 5, str(v), ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('outputs/01_sentiment_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n💾 Saved: outputs/01_sentiment_distribution.png")

**📌 Insight:** The market spent most trading days in Fear or Greed territory, with fewer Neutral periods. This creates clear opportunities to compare trader behavior across different market conditions.

### 💰 Visualization 2: Profitability by Sentiment

In [ ]:
# Calculate average daily P&L by sentiment
profitability = merged_df.groupby('classification')['total_pnl'].mean().sort_values(ascending=False)

print("💰 Average Daily P&L by Sentiment:")
print("="*60)
for sentiment, pnl in profitability.items():
    print(f"   {sentiment:15s}: ${pnl:,.2f}")

# Visualization
plt.figure(figsize=(10, 6))
profitability.plot(kind='bar', color='steelblue')
plt.title('Average Daily Profit/Loss by Market Sentiment', fontsize=14, fontweight='bold')
plt.xlabel('Sentiment Category', fontsize=12)
plt.ylabel('Average Daily P&L ($)', fontsize=12)
plt.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.7)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

# Add value labels
for i, v in enumerate(profitability):
    plt.text(i, v + (abs(v) * 0.05), f'${v:,.0f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('outputs/02_profitability_by_sentiment.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n💾 Saved: outputs/02_profitability_by_sentiment.png")

**📌 Insight:** Traders show [higher/lower] profitability during [sentiment] periods. This suggests that [interpretation based on your data - e.g., 'contrarian strategies may be effective' or 'momentum trading aligns with sentiment'].

### 📊 Visualization 3: Trading Volume by Sentiment

In [ ]:
# Calculate average daily trading volume by sentiment
volume = merged_df.groupby('classification')['trade_count'].mean().sort_values(ascending=False)

print("📊 Average Trades per Day by Sentiment:")
print("="*60)
for sentiment, count in volume.items():
    print(f"   {sentiment:15s}: {count:,.0f} trades")

# Visualization
plt.figure(figsize=(10, 6))
volume.plot(kind='bar', color='coral')
plt.title('Average Daily Trading Volume by Market Sentiment', fontsize=14, fontweight='bold')
plt.xlabel('Sentiment Category', fontsize=12)
plt.ylabel('Average Trades per Day', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

# Add value labels
for i, v in enumerate(volume):
    plt.text(i, v + (v * 0.02), f'{v:,.0f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('outputs/03_volume_by_sentiment.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n💾 Saved: outputs/03_volume_by_sentiment.png")

**📌 Insight:** Trading activity is [higher/lower] during [sentiment] periods, indicating [interpretation - e.g., 'increased market participation during greed' or 'cautious behavior during fear'].

### ⚡ Visualization 4: Leverage by Sentiment

In [ ]:
# Calculate average leverage by sentiment (if available)
if 'avg_leverage' in merged_df.columns:
    leverage = merged_df.groupby('classification')['avg_leverage'].mean().sort_values(ascending=False)
    
    print("⚡ Average Leverage by Sentiment:")
    print("="*60)
    for sentiment, lev in leverage.items():
        print(f"   {sentiment:15s}: {lev:.2f}x")
    
    # Visualization
    plt.figure(figsize=(10, 6))
    leverage.plot(kind='bar', color='darkred')
    plt.title('Average Leverage Usage by Market Sentiment', fontsize=14, fontweight='bold')
    plt.xlabel('Sentiment Category', fontsize=12)
    plt.ylabel('Average Leverage (x)', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for i, v in enumerate(leverage):
        plt.text(i, v + (v * 0.02), f'{v:.2f}x', ha='center', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('outputs/04_leverage_by_sentiment.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    print("\n💾 Saved: outputs/04_leverage_by_sentiment.png")
else:
    print("⚠️ Leverage data not available in dataset")

**📌 Insight:** Risk appetite (measured by leverage) is [higher/lower] during [sentiment] periods. This shows that traders [interpretation - e.g., 'take on more risk when optimistic' or 'become conservative during fear'].

### 📈 Visualization 5: Daily Trading Activity Trend

In [ ]:
# Sort by date for time series
merged_df_sorted = merged_df.sort_values('date')

# Create visualization
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Top: Sentiment over time
ax1.plot(merged_df_sorted['date'], merged_df_sorted['value'], color='steelblue', linewidth=1.5)
ax1.axhline(y=25, color='red', linestyle='--', alpha=0.5, label='Extreme Fear')
ax1.axhline(y=50, color='orange', linestyle='--', alpha=0.5, label='Neutral')
ax1.axhline(y=75, color='green', linestyle='--', alpha=0.5, label='Extreme Greed')
ax1.fill_between(merged_df_sorted['date'], 0, 25, alpha=0.1, color='red')
ax1.fill_between(merged_df_sorted['date'], 75, 100, alpha=0.1, color='green')
ax1.set_title('Market Sentiment Over Time', fontsize=14, fontweight='bold')
ax1.set_ylabel('Fear & Greed Index', fontsize=11)
ax1.legend(loc='upper right')
ax1.grid(alpha=0.3)

# Bottom: Trade count over time
ax2.plot(merged_df_sorted['date'], merged_df_sorted['trade_count'], color='coral', linewidth=1)
ax2.set_title('Daily Trading Volume Over Time', fontsize=14, fontweight='bold')
ax2.set_ylabel('Number of Trades', fontsize=11)
ax2.set_xlabel('Date', fontsize=11)
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/05_time_trends.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n💾 Saved: outputs/05_time_trends.png")

**📌 Insight:** The time series reveals [pattern - e.g., 'trading volume spikes align with extreme sentiment shifts' or 'consistent activity regardless of market conditions'].

---
## 7️⃣ Summary Statistics

In [ ]:
# Create comprehensive summary table
summary = merged_df.groupby('classification').agg({
    'total_pnl': ['mean', 'sum'],
    'trade_count': ['mean', 'sum'],
    'date': 'count'
}).round(2)

summary.columns = ['Avg_Daily_PnL', 'Total_PnL', 'Avg_Trades_Day', 'Total_Trades', 'Trading_Days']

print("📊 COMPREHENSIVE SUMMARY BY SENTIMENT")
print("="*80)
display(summary)

# Find best performing sentiment
best_sentiment = summary['Avg_Daily_PnL'].idxmax()
most_active = summary['Avg_Trades_Day'].idxmax()

print(f"\n🏆 Best Performance: {best_sentiment}")
print(f"📈 Most Active: {most_active}")

---
## 8️⃣ Key Insights & Recommendations

In [ ]:
print("="*80)
print("💡 KEY FINDINGS")
print("="*80)

# Automatically generate insights
best_pnl_sentiment = summary['Avg_Daily_PnL'].idxmax()
best_pnl_value = summary.loc[best_pnl_sentiment, 'Avg_Daily_PnL']

most_active_sentiment = summary['Avg_Trades_Day'].idxmax()
most_active_value = summary.loc[most_active_sentiment, 'Avg_Trades_Day']

print(f"\n1. PROFITABILITY")
print(f"   • {best_pnl_sentiment} periods show highest average daily P&L: ${best_pnl_value:,.2f}")
print(f"   • This suggests {'contrarian' if 'Fear' in best_pnl_sentiment else 'momentum'} strategies may be effective")

print(f"\n2. TRADING ACTIVITY")
print(f"   • {most_active_sentiment} periods have highest trading volume: {most_active_value:,.0f} trades/day")
print(f"   • Market participation {'increases' if 'Greed' in most_active_sentiment else 'decreases'} with {'optimism' if 'Greed' in most_active_sentiment else 'pessimism'}")

if 'avg_leverage' in merged_df.columns:
    leverage_by_sentiment = merged_df.groupby('classification')['avg_leverage'].mean()
    highest_lev = leverage_by_sentiment.idxmax()
    highest_lev_value = leverage_by_sentiment.max()
    print(f"\n3. RISK BEHAVIOR")
    print(f"   • Highest leverage during {highest_lev}: {highest_lev_value:.2f}x average")
    print(f"   • Traders take on more risk when market sentiment is {'positive' if 'Greed' in highest_lev else 'negative'}")

print("\n" + "="*80)
print("📌 BUSINESS RECOMMENDATIONS")
print("="*80)
print(f"\n• For Traders: Consider {best_pnl_sentiment.lower()} periods for optimal entry points")
print(f"• For Platforms: Implement sentiment-based alerts and risk warnings")
print(f"• For Risk Management: Monitor leverage usage during extreme sentiment shifts")
print("\n" + "="*80)

---
## ✅ Project Complete!

### 📂 Outputs Generated:
- **CSV Files:** `csv_files/merged_trading_sentiment.csv`
- **Visualizations:** 5 charts in `outputs/` folder

### 🎯 Next Steps:
1. Review all visualizations
2. Create `ds_report.pdf` using insights from this analysis
3. Update `README.md` with project summary
4. Prepare for submission!

---

**Analysis by:** Aswin  
**Date:** January 2025  
**Purpose:** Junior Data Scientist Application - PrimeTrade